In [1]:
import pandas as pd 
import numpy as np
import sklearn
import joblib
import xgboost as xgb
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import PoissonRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler

post_data = pd.read_csv('/workspaces/Crowdfunding-Social-Media-Drivers/Data/Original_dataset/processed_data.csv')

target_cols = ['likes', 'shares', 'comments', 'positive_reactions', 'negative_reactions']
features = [x for x in post_data.columns if x not in target_cols]


features = post_data[features]
targets = post_data[target_cols]



In [2]:
def pre_process(x):
    scalable_features = ['likes_at_posting', 'followers_at_posting', 'post_views']
    noticable_entities = ['ORG', 'PERSON', 'DATE',
                          'CARDINAL', 'GPE', 'PRODUCT', 
                          'WORK_OF_ART','ORDINAL', 'MONEY',
                          'TIME', 'NORP']
    scaler = RobustScaler()
    x.loc[:,scalable_features] = scaler.fit_transform(x.loc[:,scalable_features])

    x = pd.concat([x, pd.get_dummies(x.type, prefix='type', drop_first=True).astype(int)], axis=1).drop('type', axis=1)
    x = pd.concat([x, pd.get_dummies(x.page_name, prefix='page_name').astype(int)], axis=1).drop('page_name', axis=1)
    for entity in noticable_entities:
        x[f'entity_{entity}'] = x.entities_identified.fillna('None').str.split(',').apply(lambda entity_list: entity in entity_list).astype(int)
    
    x = x.drop('entities_identified', axis=1)
    return x


def process_targets(y):
    y = y.copy()
    y = np.log(y+1)
    return y

def decode_targets(y):
    y = y.copy()
    y = np.exp(y)-1
    return y

processed_features = pre_process(features)
processed_targets = process_targets(targets)
x_train, x_test, y_train, y_test = train_test_split(processed_features, processed_targets, test_size=0.2, random_state=42)

y_test = decode_targets(y_test)

In [3]:
def evaluate_model_performance(y_test, pred):
    target_cols = ['likes', 'shares', 'comments', 'positive_reactions', 'negative_reactions']
    engagement_scores = []
    for i in range(5):
        
        mse = mean_squared_error(
            y_true=y_test.values[:,i], 
            y_pred=pred[:,i]
            )
        rmse = np.sqrt(mse)


        r2 = r2_score(
            y_pred=pred[:,i],
            y_true=y_test.values[:,i]
            )
            
        msle = mean_squared_log_error(
            y_pred=pred[:,i],
            y_true=y_test.values[:,i]
            )

        ev_score = explained_variance_score(
            y_pred=pred[:,i],
            y_true=y_test.values[:,i]
        )

        engagement_scores.append([rmse, r2, msle, ev_score])

    engagement_scores = pd.DataFrame(engagement_scores, 
                                     columns=['RMSE', 'R2', 'MSLE', 'EV Score'], 
                                     index=target_cols)
    
    return engagement_scores

def save_model(model, name):
    save_path = '/workspaces/Crowdfunding-Social-Media-Drivers/Modelling/Enagement_prediction/'
    joblib.dump(model, save_path+f'/{name}.pkl')

    return None

def load_model(name):
    save_path = '/workspaces/Crowdfunding-Social-Media-Drivers/Modelling/Enagement_prediction/'
    return joblib.load(save_path+f'/{name}.pkl')
    

In [4]:
#xgb training
xgb_param_grid = {
    'n_estimators': [500, 1000, 2500, 5000],
    'learning_rate': [0.05, 0.001, 0.01, 0.1],
    'max_depth': [3, 5, 10],
    'random_state': [42],
    'n_jobs': [-1]
}


xgb_grid_search = GridSearchCV(
    xgb.XGBRegressor(),
    xgb_param_grid, cv=5, 
    scoring='neg_mean_squared_error', 
    verbose=2, n_jobs=-1
    )

xgb_grid_search.fit(x_train, y_train)

xgb_best_params = xgb_grid_search.best_params_
xgb_best_estimator = xgb_grid_search.best_estimator_

pprint(xgb_best_params)

xgb_best_estimator.fit(x_train, y_train)
save_model(xgb_best_estimator, 'xgBoost')

xgb_pred = xgb_best_estimator.predict(x_test)
xgb_pred = decode_targets(xgb_pred).astype(int)

xgb_scores = evaluate_model_performance(y_test, xgb_pred)
xgb_scores

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.5s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.5s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.6s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.6s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=500, n_jobs=-1, random_state=42; total time=   3.4s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=1000, n_jobs=-1, random_state=42; total time=   6.7s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=1000, n_jobs=-1, random_state=42; total time=   6.7s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=1000, n_jobs=-1, random_state=42; total time=   6.9s
[CV] END learning_rate=0.05, max_depth=3, n_estimators=1000, n_jobs=-1, random_state=42; total time=   7.0s
[CV] END learning_rate=0.05, max_

/home/codespace/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END learning_rate=0.05, max_depth=5, n_estimators=5000, n_jobs=-1, random_state=42; total time= 1.5min
[CV] END learning_rate=0.05, max_depth=5, n_estimators=5000, n_jobs=-1, random_state=42; total time= 1.5min
[CV] END learning_rate=0.05, max_depth=5, n_estimators=5000, n_jobs=-1, random_state=42; total time= 1.5min
[CV] END learning_rate=0.05, max_depth=10, n_estimators=500, n_jobs=-1, random_state=42; total time= 1.3min
[CV] END learning_rate=0.05, max_depth=10, n_estimators=500, n_jobs=-1, random_state=42; total time= 1.3min
[CV] END learning_rate=0.05, max_depth=5, n_estimators=5000, n_jobs=-1, random_state=42; total time= 1.5min
[CV] END learning_rate=0.05, max_depth=10, n_estimators=500, n_jobs=-1, random_state=42; total time= 1.3min
[CV] END learning_rate=0.05, max_depth=10, n_estimators=500, n_jobs=-1, random_state=42; total time= 1.2min
[CV] END learning_rate=0.05, max_depth=10, n_estimators=500, n_jobs=-1, random_state=42; total time= 1.2min
[CV] END learning_rate=0.05,

,RMSE,R2,MSLE,EV Score
likes,34.993092,0.649606,0.177561,0.659994
shares,16.251630,0.460300,0.504210,0.485186
comments,23.046288,0.622833,0.502164,0.633846
positive_reactions,17.989663,0.553123,0.466345,0.569218
negative_reactions,40.158354,0.312289,0.914257,0.335076


In [5]:
estimators = [100,500, 1000, 2000]
features = [0.33,0.5, 1, 'sqrt']
samples_leaf = [1,2,5,10]
bootstrap = [True, False]
min_samples_split = [2, 5, 10]


random_grid = {'n_estimators': estimators,
               'max_features': features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': samples_leaf,
               'bootstrap': bootstrap}

rf_grid_search = GridSearchCV(
    RandomForestRegressor(criterion="poisson"), 
    random_grid, cv=5, 
    scoring='neg_mean_squared_error', 
    verbose=2, n_jobs=-1
    )

rf_grid_search.fit(x_train, y_train)

rf_best_params = rf_grid_search.best_params_
rf_best_estimator = rf_grid_search.best_estimator_

print(rf_best_params)

rf_best_estimator.fit(x_train, y_train)
save_model(rf_best_estimator, 'RandomForest')
rf_pred = rf_best_estimator.predict(x_test)
rf_pred = decode_targets(rf_pred).astype(int)

evaluate_model_performance(y_test, rf_pred)


Fitting 5 folds for each of 384 candidates, totalling 1920 fits


ValueError: Invalid parameter 'estimator' for estimator RandomForestRegressor(criterion='poisson'). Valid parameters are: ['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].